In [1]:
!pip3 install mesa

     |████████████████████████████████| 668 kB 28.9 MB/s 
     |████████████████████████████████| 60 kB 7.7 MB/s 


In [2]:
# Importar de mesa las clases Agent y Model
from mesa import Agent, Model 

# MultiGrid para tener multiples agentes en la misma celda
from mesa.space import MultiGrid

# Los agentes se activan de manera simultanea
from mesa.time import SimultaneousActivation

# DataCollector nos ayuda a tener todo el grid completo.
from mesa.datacollection import DataCollector 

# Matplotlib nos ayuda a graficar
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# numpy y pandas se utilizan para numeros
import numpy as np
import pandas as pd

# time y datetime los utilizamos para ver el tiempo de ejecución y compararlo con nuestro parametro de tiempo maximo
import time
import datetime

In [13]:
stackedBoxes = 0
stackCounter = 0
currentStack = 0
countOfBoxes = 0
col = 0

def get_grid(model):
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for agent in cell_content:
          grid[x][y] = agent.val

    return grid

class floorAgent(Agent):
    def __init__(self, unique_id, model, isBox):
        super().__init__(unique_id, model)
        self.val = isBox
        self.partOfStack = 0 
  
class robotAgent(Agent):
      def __init__(self, unique_id, model, isRob):
        super().__init__(unique_id, model)
        self.val = isRob
        self.hasBox = 0

      def random_move(self):
        # Pick the next cell from the adjacent cells.
        next_moves = self.model.grid.get_neighborhood(self.pos, True, True)
        next_move = self.random.choice(next_moves)
        # Now move:
        self.model.grid.move_agent(self, next_move)

      def canMove(self, robPos):
          neighbours = self.model.grid.get_neighbors(
              self.pos,
              moore=True,
              include_center=True,
              )
          for neighbor in neighbours:
            if(neighbor.val == 6 and neighbor.pos == robPos):
              return False
          return True

      def placeBox(self):
        global currentStack, stackedBoxes, countOfBoxes
        self.hasBox = 0
        b = floorAgent(self.pos, self, countOfBoxes+1)
        b.partOfStack = 1
        self.model.grid.place_agent(b, self.pos)
        countOfBoxes+=1
        stackedBoxes+=1
        if countOfBoxes == 5:
          currentStack += 1
          countOfBoxes = 0
        
      def moveToStack(self):
        global currentStack
        newPosx = self.pos[0]
        newPosy = self.pos[1]
        if self.pos[0] != 0 and self.canMove((self.pos[0] - 1, self.pos[1])):
          newPosx = self.pos[0] - 1
          self.model.grid.move_agent(self, (newPosx, newPosy))
        if self.pos[1] != currentStack and self.canMove((self.pos[0], self.pos[1] - 1)) and self.pos[0] == 0:
          newPosy = self.pos[1] - 1
          self.model.grid.move_agent(self, (newPosx, newPosy))
        if self.pos[0] == 0 and self.pos[1] == currentStack:
          self.placeBox()

      def step(self):
          neighbours = self.model.grid.get_neighbors(
              self.pos,
              moore=True,
              include_center=True,
              )
          for neighbor in neighbours:
            if neighbor.val == 1 and neighbor.partOfStack == 0 and self.hasBox == 0:
              self.hasBox = 1
              self.model.grid.remove_agent(neighbor)

            #si estoy en 0,currentStack y tengo una caja, deposita la caja
            '''
            if self.pos[0] == 0 and self.pos[1] == currentStack and self.hasBox == 1:
              b = floorAgent(self.pos, self, 1)              
              self.model.grid.place_agent(b, self.pos)
              self.hasBox = 0
              countOfBoxes+=1
              stackedBoxes+=1
              if countOfBoxes == 5:
                currentStack += 1
                countOfBoxes = 0
            '''

      def advance(self):
          #if i have a box, go deposit, else random
            if self.hasBox == 1:
              self.moveToStack()
            else:
              self.random_move()
          
class robotStackModel(Model):
    def __init__(self, width, height, numBoxes, numAgents):
        self.num_agents = width * height
        self.grid = MultiGrid(width, height, True)
        self.schedule = SimultaneousActivation(self)

        self.numBoxes = numBoxes
        agentsList = np.zeros((width, height),int)
        x = 0
        j = 0
        while x <= numBoxes and j < numAgents:
          posx = np.random.randint(0,width-1)
          posy = np.random.randint(0,height-1)
          posxR = np.random.randint(0,width-1)
          posyR = np.random.randint(0,height-1)
          if agentsList[posx][posy] == 0 and agentsList[posxR][posyR] == 0 and x != (numBoxes):
            agentsList[posx][posy] = 1
            x+=1
          if agentsList[posx][posy] == 0 and agentsList[posxR][posyR] == 0 and j!= numAgents:
            agentsList[posxR][posyR] = 6
            j+=1

        print(agentsList)

        for (content, x, y) in self.grid.coord_iter():
          if(agentsList[x][y]) == 6:
            a = robotAgent((x, y), self, agentsList[x][y])
            self.grid.place_agent(a, (x, y))
            self.schedule.add(a)

          else:  # Si el valor de agentsList es 1, la variable .val de la clase floorAgent sera 1, osea una caja.
            b = floorAgent((x, y), self, agentsList[x][y])
            self.grid.place_agent(b, (x, y))
            self.schedule.add(b)            
        
        # Aquí definimos con colector para obtener el grid completo.
        self.datacollector = DataCollector(
            model_reporters={"Grid": get_grid})
    
    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

    def stacked(self):
      global stackedBoxes
      if stackedBoxes == self.numBoxes:
        self.step()
        return True
      return False

In [14]:
GRID_SIZE = 10

NUM_BOXES = 40

NUM_AGENTS = 5

MAX_RUN = 0.6

movimientos = 0

# Registramos el tiempo de inicio y corremos el modelo
start_time = time.time()
model = robotStackModel(GRID_SIZE, GRID_SIZE, NUM_BOXES, NUM_AGENTS)

while(time.time()-start_time) < MAX_RUN and model.stacked() == False:
  movimientos += 1
  model.step()
# Imprimimos el tiempo que le tomó correr al modelo.
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))

print('Movimientos totales para completar el ordenamiento: ', movimientos)

[[0 1 6 1 0 0 0 0 1 0]
 [1 1 0 1 0 1 0 0 1 0]
 [1 1 1 1 0 0 1 0 0 0]
 [1 1 1 0 1 1 1 6 1 0]
 [1 1 0 0 1 1 6 0 1 0]
 [0 0 0 0 1 0 1 1 1 0]
 [0 1 1 0 1 1 0 0 0 0]
 [0 0 0 6 1 1 0 0 0 0]
 [6 1 0 1 1 0 1 0 1 0]
 [0 0 0 0 0 0 0 0 0 0]]
Tiempo de ejecución: 0:00:00.035680
Movimientos totales para completar el ordenamiento:  112


In [15]:
all_grid = model.datacollector.get_model_vars_dataframe()

In [16]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=plt.cm.Greens)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=movimientos)

In [17]:
anim

# Analiza si existe una estrategia que podría disminuir el tiempo dedicado, así como la cantidad de movimientos realizados. ¿Cómo sería? Descríbela

Lo que se me ocurre es que las posiciones de las cajas se las pasemos a los robots y ya desde un inicio, en vez de irse a mover random, que vayan a donde estan las cajas y las recogan. Esto haria que encontrar una caja no sea cuestion de suerte si no de tiempo calculado.